In [3]:
import pandas as pd
import numpy as np
import re

In [2]:
file1 = open('train.txt', 'r') 
lines = file1.readlines()

In [10]:
ids = []
revs = []
comps = []
txts = []

for line in lines:
    ids.append(line[1:5])
    revs.append(line[6:9])
    comps.append(line[10:13])
    txts.append(line[15:])

In [12]:
df = pd.DataFrame({"Avis" : revs, "Entreprise" : comps, "Tweet" : txts}, index = ids)
df.head(10)

,Avis,Entreprise,Tweet
0000,neu,apl,20 min line @apple store @short pump.\n
0001,irr,msf,Nueva tecnología convierte cualquier superfici...
0002,neu,ggl,Some people should not post replies in #Google...
0003,neg,apl,I know a few others having same issue RT @Joel...
0004,neg,msf,"#Microsoft - We put the """"backwards"""" into bac..."
0005,neg,twt,#twitter is sooo trash ritenow with all dezz #...
0006,neu,apl,RT @jesperordrup: Hi @apple. Household has 4 i...
0007,irr,msf,#ALG Culminando formación en #Microsoft M-2778...
0008,neu,msf,#Microsoft Community Blogs The 7/365 Review - ...
0009,irr,twt,Buenas noches a todos #Twitter off\n


In [13]:
df.Avis.unique()

array(['neu', 'irr', 'neg', 'pos'], dtype=object)

In [14]:
df.Entreprise.unique()

array(['apl', 'msf', 'ggl', 'twt'], dtype=object)

In [15]:
len(df)

4173

In [17]:
df.groupby(by="Avis").count()

,Entreprise,Tweet
Avis,,
irr,1389,1389
neg,486,486
neu,1871,1871
pos,427,427


In [19]:
df.groupby(by=["Entreprise", "Avis"]).count()

Tweet
Entreprise Avis       
apl        irr     125
           neg     267
           neu     422
           pos     140
ggl        irr     398
           neg      47
           neu     455
           pos     167
msf        irr     401
           neg     111
           neu     516
           pos      73
twt        irr     465
           neg      61
           neu     478
           pos      47